In [ ]:
import pandas as pd
import plotly.express as px
import requests
import json
import datetime
import concurrent.futures
import swifter
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from numpy import random
import math
import seaborn as sns
plt.style.use('fivethirtyeight')
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
import dateutil
import mariadb
import re

In [ ]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [ ]:
try:
    mydb = mariadb.connect(host="172.17.0.3", database = 'dados_tribunais',user="root", passwd="abc@123")
    query = "select * from processos where grau='JE' and tribunal='TJRN' and orgao_julgador = '4º JUIZADO ESPECIAL DA FAZENDA PÚBLICA';"
    df_tribunal = pd.read_sql(query,mydb)
    mydb.close() #close the connection
except Exception as e:
    mydb.close()
    print(str(e))

In [ ]:
df_tribunal.shape

In [ ]:
df_tribunal.head(1)

In [ ]:
df_tribunal.drop_duplicates(inplace=True)
#df_tribunal.drop(columns=['sort', 'codigo', 'classe', 'numero_processo', 'orgao_julgador'], inplace=True, errors='ignore')
if df_tribunal['assuntos'].isna().sum() > (0.8 * len(df_tribunal['assuntos'])):
    df_tribunal.drop(columns=['assuntos'], inplace=True)

In [ ]:
movimentacoes = list(df_tribunal.movimentos.values)
print(len(movimentacoes))

In [ ]:
tribunal = 'TRF'
if tribunal[:2] == 'TJ':
    movimentacoes_gabinete = pd.read_csv('dados/movimentos_gabinete.csv', sep=';', header='infer')
    movimentacoes_gabinete['codigo'] = movimentacoes_gabinete['vazio.1']
    lista_movimentacoes_gabinete = movimentacoes_gabinete['Código'].to_list()
    movimentacoes_secretaria = pd.read_csv('dados/movimentos_secretaria.csv', sep=';', header='infer')
    lista_movimentacoes_secretaria = movimentacoes_secretaria['codigo'].to_list()
else:
    movimentacoes_gabinete = pd.read_csv('dados/jf_movimentos_gabinete.csv', sep=';', header='infer')
    lista_movimentacoes_gabinete = movimentacoes_gabinete['codigo'].to_list()
    movimentacoes_secretaria = pd.read_csv('dados/jf_movimentos_secretaria.csv', sep=';', header='infer')
    lista_movimentacoes_secretaria = movimentacoes_secretaria['codigo'].to_list()

lista_movimentacoes_gabinete_set = set(lista_movimentacoes_gabinete)

In [ ]:
padrao_movimentacao = r'\[(\d+),\s+\'(.*?)\',\s+\'(\d{4}-\d{2}-\d{2})\'\]'

def calcular_data_sentenca(movimentacao):
    resultado = re.findall(padrao_movimentacao, movimentacao)
    if resultado:
        for codigo, texto, data in resultado:
            if int(codigo) in lista_movimentacoes_gabinete_set:
                #print(codigo, '\: ', data)
                return data
        
    return -1

In [ ]:
df_tribunal['data_sentenca'] = df_tribunal['movimentos'].swifter.apply(calcular_data_sentenca)

In [ ]:
df_tribunal.head()

In [ ]:
# Pegando apenas o código do primeiro assunto
df_tribunal['assuntos'] = df_tribunal['assuntos'].swifter.apply(lambda x: x.split(',')[0].split(':')[1].strip())

In [ ]:
from datetime import datetime

def calcular_tempo_entre_movimentacoes(movimentacoes, inicial, final):
    if final < inicial:
        return -1
    
    padrao_movimentacao = r'\[(\d+),\s+\'(.*?)\',\s+\'(\d{4}-\d{2}-\d{2})\'\]'
    resultado = re.findall(padrao_movimentacao, movimentacoes)
    try:
        if len(resultado) >= 2 and len(resultado) > final:
            data_1 = datetime.strptime(resultado.pop(inicial)[2], '%Y-%m-%d')
            data_2 = datetime.strptime(resultado.pop(inicial)[2], '%Y-%m-%d')
                       
            return abs((data_1 - data_2).days)
        else:
            return -1
    except:
        print(f'erro: {len(resultado)}, {resultado}')
    return -1


In [ ]:
def calcular_tempo_ate_sentenca(df):

    #print(abs(datetime.strptime(df['data_sentenca'], '%Y-%m-%d') - (datetime.strptime(df['data_ajuizamento'],'%Y-%m-%d').days)))
    return abs(datetime.strptime(df['data_sentenca'], '%Y-%m-%d') - (datetime.strptime(df['data_ajuizamento'],'%Y-%m-%d').days))

In [ ]:
def estatisticas_movimentacoes_processo(movimentacoes):
    min = np.inf
    max = 0
    media = 0.0
    lista_mov_processos = list()
    for movimento in movimentacoes:
        print(movimento)
        lista_mov_processos.append(len(movimento)/3)
        if len(movimento)/3 > max:
            max = len(movimento)/3
        elif len(movimento)/3 < min:
            min = len(movimento)/3
        media += len(movimento)/3
    media = media / len(movimentacoes)
    return media, lista_mov_processos

In [ ]:
df_tribunal['tempo_entre_1e2_mov'] = df_tribunal['movimentos'].swifter.apply(calcular_tempo_entre_movimentacoes, args=(0,1))
df_tribunal['tempo_entre_2e3_mov'] = df_tribunal['movimentos'].swifter.apply(calcular_tempo_entre_movimentacoes, args=(1,2))
df_tribunal['tempo_entre_3e4_mov'] = df_tribunal['movimentos'].swifter.apply(calcular_tempo_entre_movimentacoes, args=(2,3))

df_tribunal['mais60d'] = (df_tribunal['tempo_entre_1e2_mov'] > 60) | (df_tribunal['tempo_entre_2e3_mov'] > 60) | (df_tribunal['tempo_entre_3e4_mov'] > 60) #| (df_tribunal['tempo_entre_4e5_mov'] > 60)
#df_tribunal['mais60d'].replace(['False','True'],[0,1],inplace=True)

In [ ]:
print(df_tribunal['mais60d'].value_counts())

In [ ]:
df_sentenciados = df_tribunal[df_tribunal['data_sentenca'] != -1]
df_sentenciados['data_ajuizamento'] = pd.to_datetime(df_sentenciados['data_ajuizamento'], errors='coerce')
df_sentenciados['data_sentenca'] = pd.to_datetime(df_sentenciados['data_sentenca'], errors='coerce')
df_sentenciados['tempo_ate_sentenca'] = (df_sentenciados['data_sentenca'] - df_sentenciados['data_ajuizamento']).dt.days
print(df_sentenciados['tempo_ate_sentenca'].describe())
df_sentenciados['tempo_ate_sentenca'] = df_sentenciados['tempo_ate_sentenca'].apply(lambda x: x if pd.notnull(x) else -1)


In [ ]:
df_nao_sentenciados = pd.DataFrame()
mascara = df_tribunal['data_sentenca'].values == -1
df_nao_sentenciados = df_tribunal[mascara]
df_nao_sentenciados['tempo_ate_sentenca'] = -1

In [ ]:
df_nao_sentenciados.head()

In [ ]:
print(df_sentenciados.shape)
print(df_nao_sentenciados.shape)
print(df_tribunal.shape)

In [ ]:
print(df_sentenciados['tempo_ate_sentenca'].info())

In [ ]:
df_tribunal = pd.concat([df_sentenciados, df_nao_sentenciados])
#print(df_tribunal.head(1))

In [ ]:
df_tribunal.columns

In [47]:
## data_sentenca | tempo_ate_sentenca | tempo_entre_1e2_mov | tempo_entre_2e3_mov | tempo_entre_3e4_mov | mais60d

try:
    mydb = mariadb.connect(host="172.17.0.3", database='dados_tribunais', user="root", passwd="abc@123")

    mycursor = mydb.cursor()
    for processo in df_tribunal.numero_processo.to_list():
        sentenca = df_tribunal[df_tribunal.numero_processo == processo].data_sentenca.values[0]
        tempo_ate_sentenca = df_tribunal[df_tribunal.numero_processo == processo].tempo_ate_sentenca.values[0]
        tempo_entre_1e2_mov = df_tribunal[df_tribunal.numero_processo == processo].tempo_entre_1e2_mov.values[0]
        tempo_entre_2e3_mov = df_tribunal[df_tribunal.numero_processo == processo].tempo_entre_2e3_mov.values[0]
        tempo_entre_3e4_mov = df_tribunal[df_tribunal.numero_processo == processo].tempo_entre_3e4_mov.values[0]
        mais60d = df_tribunal[df_tribunal.numero_processo == processo].mais60d.values[0]
        mycursor.execute(f"UPDATE processos SET data_sentenca = '{sentenca}', tempo_ate_sentenca = '{tempo_ate_sentenca}', tempo_entre_1e2_mov = '{tempo_entre_1e2_mov}', tempo_entre_2e3_mov = '{tempo_entre_2e3_mov}', tempo_entre_3e4_mov = '{tempo_entre_3e4_mov}', mais60d = {mais60d} WHERE numero_processo = '{processo}';")
    mydb.commit()
    mydb.close()  # close the connection
except Exception as e:
    mydb.close()
    print(str(e))

In [ ]:
# tribunal = df_tribunal['tribunal'][0]
# grau = df_tribunal['grau'][0]
# data_inicio = df_tribunal['data_ajuizamento'].min().date().strftime("%Y-%m-%d")
# data_final = df_tribunal['data_ajuizamento'].max().date().strftime("%Y-%m-%d")
# classe = df_tribunal['codigo_classe'][0]
# nome_dataset = tribunal + '_' + grau + '_' + str(classe) + '_' + str(data_inicio) + '-' + str(data_final)
# df_tribunal.to_csv(f'dados/processados/{nome_dataset}.csv', sep=';', header=True, index=False, compression='zip')
# nome_dataset = tribunal + '_' + grau + '_' + str(classe) + '_' + str(data_inicio) + '-' + str(data_final) + '_' + 'sentenciados'
# df_sentenciados.to_csv(f'dados/processados/{nome_dataset}.csv', sep=';', header=True, index=False, compression='zip')
# nome_dataset = tribunal + '_' + str(classe) + '_' + str(data_inicio) + '-' + str(data_final) + '_' + 'nao_sentenciados'
# df_nao_sentenciados.to_csv(f'dados/processados/{nome_dataset}.csv', sep=';', header=True, index=False, compression='zip')

## EDA

In [ ]:
df_tribunal.isna().sum()

In [ ]:
df_tribunal.info()

In [ ]:
#print(df_sentenciados.shape[0], df_nao_sentenciados.shape[0])
try:
    percentual_sentenciados = (int(df_sentenciados.shape[0])/int(df_tribunal.shape[0]) * 100)
    percentual_sentenciados = float("{:.4f}".format(percentual_sentenciados))
    print(f'Percentual da base de casos sentenciados: {percentual_sentenciados}%')
except:
    pass

In [ ]:
fig = px.histogram(df_sentenciados, x='data_ajuizamento', marginal='rug', title='Distribuição dos processos já sentenciados pela data de ajuizamento')
fig.show()

In [ ]:
fig = px.histogram(df_nao_sentenciados, x='data_ajuizamento', marginal='rug', title='Distribuição dos processos ainda não sentenciados pela data de ajuizamento')
fig.show()

In [ ]:
fig = px.histogram(df_sentenciados, x='tempo_ate_sentenca', marginal='rug', title='Distribuição do tempo até a sentença')
fig.show()

In [ ]:
media, lista_mov_processos = estatisticas_movimentacoes_processo(df_sentenciados['movimentos'].to_list())
print(f'Média de movimentações por processo: {media}')
fig = px.histogram(lista_mov_processos, marginal='rug', title='Distribuição do número de movimentações por processo',
                   nbins=math.ceil((media *2)), labels={'x':'Movimentações', 'y':'Ocorrências'}).update_layout(
    yaxis_title="Ocorrências", xaxis_title="Movimentações")
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y=df_sentenciados['tempo_entre_2e3_mov'], 
                     name='2ª e 3ª movimentação', marker_color = 'indianred'))
fig.add_trace(go.Box(y=df_sentenciados['tempo_entre_3e4_mov'], 
                     name='3ª e 4ª movimentação', marker_color = 'lightseagreen'))
fig.update_layout(title='Distribuição do tempo necessário para realizar as duas primeiras movimentações processuais')
fig.show()


In [ ]:
#df_sentenciados.groupby(['data_sentenca'])['tempo_ate_sentenca'].count()
df_ordenado = df_sentenciados.sort_values(by='data_sentenca')
fig = px.line(df_ordenado, x='data_sentenca', y='tempo_ate_sentenca', title='Tempo até sentença ao longo dos anos').update_layout(
    yaxis_title="Tempo até sentença", xaxis_title="Ano")
fig.show()

In [ ]:
top99_sentenciados = df_sentenciados['tempo_ate_sentenca'].quantile(0.80)
print(f'(80% dos processos são sentenciados em até {top99_sentenciados:.1f} dias)')

In [ ]:
sns.set()
fig = sns.pairplot(df_sentenciados[['tempo_entre_1e2_mov', 'tempo_entre_2e3_mov', 
                             'tempo_entre_3e4_mov', 'mais60d', 'tempo_ate_sentenca']], size = 2.5)
plt.show()

In [ ]:
sns.heatmap(df_sentenciados[['tempo_entre_1e2_mov', 'tempo_entre_2e3_mov', 
                             'tempo_entre_3e4_mov', 'mais60d', 'tempo_ate_sentenca']].corr(),annot=True,cmap='RdYlGn',linewidths=0.2)
fig=plt.gcf()
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
def lista_movimentacoes(movimentacoes, posicao):
    movimentacoes_posicao_3 = list()
    for movimento in movimentacoes:
        movimentacoes_posicao_3.append(movimento[posicao -1][0])
    return list(set(movimentacoes_posicao_3))

    
mais_demoradas_movimentacoes_pos_3 = lista_movimentacoes(df_sentenciados['movimentos'].to_list(), 3)
print(mais_demoradas_movimentacoes_pos_3)


for codigo in mais_demoradas_movimentacoes_pos_3:
    acoes_secretaria = 0
    acoes_gabinete = 0
    acoes_sentenca = 0
    if int(codigo) in lista_movimentacoes_secretaria:
        acoes_secretaria += 1
    elif int(codigo) in lista_movimentacoes_gabinete:
        acoes_gabinete += 1
    elif int(codigo) in lista_movimentacoes_gabinete:
        acoes_sentenca += 1

print(f'Das movimentações mais demoradas, {acoes_secretaria} são realizadas pela secretaria, {acoes_gabinete} pelo gabinete e {acoes_sentenca} são sentenças')
        

## Testes com ML

In [ ]:
print(df_tribunal.columns)

In [ ]:
df_ml = df_sentenciados.drop(columns=['movimentos', 'numero_processo', 'data_ajuizamento', 'ultima_atualizacao', 'codigo', 'data_sentenca','tribunal', 'orgao_julgador', 'municipio', 'grau'])
df_ml.dropna(inplace=True)
df_ml = pd.get_dummies(df_ml, columns=['assuntos', 'classe'], drop_first=True)
df_x = df_ml.drop(columns=['tempo_ate_sentenca', 'mais60d'])
#print(df_ml.head(1))
print(df_ml.columns)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold

metricas = ['accuracy', 'f1', 'precision', 'recall']
rfc = RandomForestClassifier(random_state=19, n_jobs=-1)
adc = AdaBoostClassifier(random_state=19,algorithm="SAMME", n_estimators=200)
mlp = MLPClassifier(random_state=19)
dtc = DecisionTreeClassifier(random_state=19)

scores_rfc = cross_validate(rfc, df_x,
                             df_ml['mais60d'], cv=10, scoring=metricas)
scores_adc = cross_validate(adc, df_x,
                             df_ml['mais60d'], cv=10, scoring=metricas)
scores_mlp = cross_validate(mlp, df_x,
                             df_ml['mais60d'], cv=10, scoring=metricas)
scores_dtc = cross_validate(dtc, df_x,
                             df_ml['mais60d'], cv=10, scoring=metricas)

In [ ]:
chaves = scores_dtc.keys()
print(f'Usando as colunas {df_x.columns} chegou-se ao seguinte resultado\n\n')
print(f'Métrica \tRandomForest \tAdaBoost \tMLP \tDecisionTree')
for chave in chaves:
    print(f'{chave}:\t {scores_rfc[chave].mean():.3f} \t\t{scores_adc[chave].mean():.3f} \t\t{scores_mlp[chave].mean():.3f} \t\t{scores_dtc[chave].mean():.3f}')


### Modelo tentando prever se vai atrasar ou não

In [ ]:
from sklearn.tree import export_text
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_x, df_ml['mais60d'],test_size=0.20, random_state=19)

clf = DecisionTreeClassifier(max_depth=10).fit(X_train, y_train)
print(clf.score(X_test,y_test))
r = export_text(clf, feature_names=df_x.columns)

print(r)
 

### Regressão para o tempo até a sentença

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_x, 
                                                    df_ml['tempo_ate_sentenca'], test_size=0.20, random_state=19)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


model = RandomForestRegressor(max_depth=100, random_state=0, n_estimators=500)
model.fit(X_train, y_train)
rf_y_pred = model.predict(X_test)

rf_r2 = r2_score(y_test, rf_y_pred)
rf_mae = mean_absolute_error(y_test, rf_y_pred)
rf_rmse = mean_squared_error(y_test, rf_y_pred, squared=False)
rf_mse = mean_squared_error(y_test, rf_y_pred)

print('----------------------------------------------------')

print(f'AdaBoost Regressor')
regr = AdaBoostRegressor(random_state=0, n_estimators=100)
regr.fit(X_train, y_train)
ada_y_pred = regr.predict(X_test)

ada_r2 = r2_score(y_test, ada_y_pred)
ada_mae = mean_absolute_error(y_test, ada_y_pred)
ada_rmse = mean_squared_error(y_test, ada_y_pred, squared=False)
ada_mse = mean_squared_error(y_test, ada_y_pred)

print('----------------------------------------------------')

print(f'MLP Regressor')

regr_mlp = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)
mlp_y_pred = regr_mlp.predict(X_test)

mlp_r2 = r2_score(y_test, mlp_y_pred)
mlp_mae = mean_absolute_error(y_test, mlp_y_pred)
mlp_rmse = mean_squared_error(y_test, mlp_y_pred, squared=False)
mlp_mse = mean_squared_error(y_test, mlp_y_pred)

print('----------------------------------------------------')

print(f'DT Regressor')
regressor = DecisionTreeRegressor(random_state=0).fit(X_train, y_train)
dt_y_pred = regr_mlp.predict(X_test)

dt_r2 = r2_score(y_test, dt_y_pred)
dt_mae = mean_absolute_error(y_test, dt_y_pred)
dt_rmse = mean_squared_error(y_test, dt_y_pred, squared=False)
dt_mse = mean_squared_error(y_test, dt_y_pred)

print(f'Metric\t\t\t\t RF\t AdaBoost\t MLP\t\t DT')
print(f"Mean Squared Error:\t\t {rf_r2:.3f}\t {ada_r2:.3f}\t\t {mlp_r2:.3f}\t\t {dt_r2:.3f}")
print(f"R-squared (R²):\t\t\t {rf_mae:.3f}\t {ada_mae:.3f}\t {mlp_mae:.3f}\t\t {dt_mae:.3f}")
print(f"Mean Absolute Error (MAE):\t {rf_mae:.3f}\t {ada_rmse:.3f}\t {mlp_rmse:.3f}\t {dt_rmse:.3f}")
print(f"Root Mean Squared Error (RMSE):\t {rf_rmse:.3f} {ada_mse:.3f}\t {mlp_mse:.3f}\t {dt_mse:.3f}")